<a href="https://colab.research.google.com/github/ppaulo17/Mine/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  #!pip install tf-nightly
  pass
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])

train_df['type'] = train_df['type'].map({'ham': 0, 'spam': 1})
test_df['type'] = test_df['type'].map({'ham': 0, 'spam': 1})

train_labels = train_df.pop('type')
test_labels = test_df.pop('type')

encoder = tf.keras.layers.TextVectorization(max_tokens=1000, output_sequence_length=100)
encoder.adapt(train_df['msg'].values)

In [ ]:
model=tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True
    ),

    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_df['msg'].values,
    train_labels.values,
    epochs=10,
    validation_data=(test_df['msg'].values, test_labels),
    verbose=2
)



In [ ]:
def predict_message(pred_text):
  prediction = model.predict(tf.constant([pred_text]))[0][0]

  label = "ham" if prediction < 0.5 else "spam"


  return [prediction, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
